Hi Rob, can you please get me the genetic distances of paralogous pairs found in my project folder: ~NIexpression/data/gene_duplicates/paralogs.csv? Thanks!





In [26]:
from annotation import Transcript, GFF_line
from Bio import SeqIO, AlignIO
import glob, re, itertools, gzip


from Bio.Align.Applications import MuscleCommandline
from ness_fasta import ness_fasta
import signal, gzip
from Bio import AlignIO
from Bio.Seq import Seq
import subprocess
from Bio.SeqRecord import SeqRecord

def signal_handler(signum, frame):
    raise Exception("Timed out!")
from Bio import SeqIO
import sys, subprocess, os, glob, re, Bio, random
from Bio import AlignIO, SeqIO
from Bio.Align import AlignInfo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict
#from Bio.Alphabet import IUPAC, Gapped
#from Bio.Align.Generic import Alignment
from Bio.Align import Alignment
from Bio.Align import MultipleSeqAlignment
#alphabet = Gapped(IUPAC.ambiguous_dna)

from math import log
from glob import glob
from Bio import SeqIO, AlignIO, codonalign
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment


def translation_aligner(seqfile, prefix, write=True):
    #Pass fasta file
    #Select the desired NCBI translation table, 1 = standard (http://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi)
    translationTable = 1
    # take list of sequences and read the fasta sequences into a dictionary
    seq_dict = SeqIO.to_dict(SeqIO.parse(open(seqfile), 'fasta'))
    # Translate the sequences
    aa_seqs = []
    for key in seq_dict:
        aaSeq = SeqRecord(seq_dict[key].seq.translate(table=translationTable), id=key)
        aa_seqs.append(aaSeq)
    # Replace stop codons with X (unknown aa) so muscle doesn't drop them
    for aaSeq in aa_seqs:
        noStopCodonSeq = str(aaSeq.seq).replace('*', 'X')
        aaSeq.seq = Seq(noStopCodonSeq)
    # write aa seq to file
    o = open("tmp.aa.unALN.fasta", 'w')
    SeqIO.write(aa_seqs, o, 'fasta')
    o.close()
    # do the alignment
    cmd = "muscle -align tmp.aa.unALN.fasta -output tmp.aa.ALN.fasta"  
    child = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr = subprocess.PIPE, shell=True) #don't pipe stderr or muscle hangs
    stdout, stderr = child.communicate()
    aaAlignment = AlignIO.read("tmp.aa.ALN.fasta", 'fasta')
    #Convert the aa alignment into a dna alignment
    dnaAlignment = MultipleSeqAlignment([])
    for aa_seq in aaAlignment:
        aaCount = 0
        dnaSeq = ''
        for aa in aa_seq.seq:
            if (aa == '-'):
                dnaSeq +=  '---'
            else:
                dnaSeq += seq_dict[aa_seq.id].seq[aaCount*3:aaCount*3+3]
                aaCount+=1
        dnaAlignment.append(SeqRecord(Seq(str(dnaSeq)), id = aa_seq.id))
    if write == True:
        o = open(prefix + '.aln.fasta', 'w')
        AlignIO.write([dnaAlignment], o, 'fasta')
        o.close()
    return dnaAlignment


def degenerate(seq):
    degen_seq = ''
    for idx in range(0, len(seq)-2,3):    
        codon = seq[idx:idx+3]
        try:
            degen  = degen_dict[codon.lower()]
        except KeyError:
            degen = 'nnn'
        degen_seq = degen_seq+degen
    return degen_seq


def JCdistance(diffs, seq_length):
    """ 
    distance = -b log(1 - p / b)
    where:
    b = 3/4
    and p = p-distance, diffs/seq_length i.e. uncorrected distance between sequences 
    """
    b = 0.75
    try: p = float(diffs)/seq_length
    except ZeroDivisionError:
        return None
    try: K = -b * log( 1-p/b )
    except ValueError: 
           return None
    return K


def k0k4(translation_alignment_file):
    with open(translation_alignment_file,'r') as f:
        nuc_aln = AlignIO.read(f, 'fasta')
        nuc1 = nuc_aln[0]
        nuc2 = nuc_aln[1]
        degen_map = degenerate(str(nuc1.seq))
        data = {'diffs':{'0':0,'2':0,'4':0}, 'sites':{'0':0,'2':0,'4':0}}
        for idx in range(len(degen_map)):
            d, base1, base2 = [thing[idx] for thing in [degen_map, str(nuc1.seq), str(nuc2.seq)]]
            if base1 in 'ATGC' and base2 in 'ATGC' and d in '024':
                data['sites'][d]+=1
                if base1 != base2:
                    data['diffs'][d]+=1
        k0 = JCdistance(data['diffs']['0'], data['sites']['0'])
        k4 = JCdistance(data['diffs']['4'], data['sites']['4'])
        total_sites = sum([data['sites'][i] for i in data['sites']])
        return(k0,k4,total_sites)

        
        
degen_dict = {
    'ttt': '002', 'ttc': '002', 'tta': '202', 'ttg': '202', \
    'tct': '004', 'tcc': '004', 'tca': '004', 'tcg': '004', \
    'tat': '002', 'tac': '002', 'taa': '022', 'tag': '002', \
    'tgt': '002', 'tgc': '002', 'tga': '020', 'tgg': '000', \
    'ctt': '004', 'ctc': '004', 'cta': '204', 'ctg': '204', \
    'cct': '004', 'ccc': '004', 'cca': '004', 'ccg': '004', \
    'cat': '002', 'cac': '002', 'caa': '002', 'cag': '002', \
    'cgt': '004', 'cgc': '004', 'cga': '204', 'cgg': '204', \
    'att': '003', 'atc': '003', 'ata': '003', 'atg': '000', \
    'act': '004', 'acc': '004', 'aca': '004', 'acg': '004', \
    'aat': '002', 'aac': '002', 'aaa': '002', 'aag': '002', \
    'agt': '002', 'agc': '002', 'aga': '202', 'agg': '202', \
    'gtt': '004', 'gtc': '004', 'gta': '004', 'gtg': '004', \
    'gct': '004', 'gcc': '004', 'gca': '004', 'gcg': '004', \
    'gat': '002', 'gac': '002', 'gaa': '002', 'gag': '002', \
    'ggt': '004', 'ggc': '004', 'gga': '004', 'ggg': '004'
}

codons = {
    "TTT": "F", "TTC": "F", "TTA": "L", "TTG": "L",
    "TCT": "S", "TCC": "S", "TCA": "S", "TCG": "S",
    "TAT": "Y", "TAC": "Y", "TAA": "*", "TAG": "*",
    "TGT": "C", "TGC": "C", "TGA": "*", "TGG": "W",
    "CTT": "L", "CTC": "L", "CTA": "L", "CTG": "L",
    "CCT": "P", "CCC": "P", "CCA": "P", "CCG": "P",
    "CAT": "H", "CAC": "H", "CAA": "Q", "CAG": "Q",
    "CGT": "R", "CGC": "R", "CGA": "R", "CGG": "R",
    "ATT": "I", "ATC": "I", "ATA": "I", "ATG": "M",
    "ACT": "T", "ACC": "T", "ACA": "T", "ACG": "T",
    "AAT": "N", "AAC": "N", "AAA": "K", "AAG": "K",
    "AGT": "S", "AGC": "S", "AGA": "R", "AGG": "R",
    "GTT": "V", "GTC": "V", "GTA": "V", "GTG": "V",
    "GCT": "A", "GCC": "A", "GCA": "A", "GCG": "A",
    "GAT": "D", "GAC": "D", "GAA": "E", "GAG": "E",
    "GGT": "G", "GGC": "G", "GGA": "G", "GGG": "G",
    "---": "-"}

In [13]:

CINC_CDS = {}
for s in SeqIO.parse('/research/projects/chlamydomonas/NIexpression/genome_info/Cincerta/longest_isoform_transcript.fa', 'fasta'):
    k = s.id.split(".")[0]
    CINC_CDS[k] = s
    
CSCH_CDS = {}
for s in SeqIO.parse('/research/projects/chlamydomonas/NIexpression/genome_info/Cschloesseri/longest_isoform_transcript.fa', 'fasta'):
    k = s.id.split(".")[0]
    CSCH_CDS[k] = s
    
MALA_CDS = {}
for s in SeqIO.parse('/research/projects/chlamydomonas/NIexpression/genome_info/Malaysian/longest_isoform_transcript.fa', 'fasta'):
    k = s.id.split(".")[0]
    MALA_CDS[k] = s


JG05_CDS = {}
for s in SeqIO.parse('/research/projects/chlamydomonas/NIexpression/genome_info/JG5/longest_isoform_transcript.fa', 'fasta'):
    k = s.id.split(".")[0]
    JG05_CDS[k] = s
    
CREN_CDS = {}
for s in SeqIO.parse(gzip.open('/research/references/chlamydomonas/6.1_chlamy/annotation/CreinhardtiiCC_4532_707_v6.1.cds_primaryTranscriptOnly.fa.gz', 'rt'), 'fasta'):
    k = s.id.split("_4532.1")[0]
    CREN_CDS[k] = s

CDSs = {'CINC': CINC_CDS,
        'CREN': CREN_CDS,
        'CSCH': CSCH_CDS,
        'CMAL': MALA_CDS,
        'JG5': JG05_CDS,
       }


In [16]:
paralog_files = glob.glob("../data/gene_duplicates/*_paralogousgenepairs*")

In [32]:
mkdir ../data/gene_duplicates/translation_alignments/

mkdir: cannot create directory ‘../data/gene_duplicates/translation_alignments/’: Permission denied


In [47]:
bc = 0

for paralog_file in paralog_files:
    species = paralog_file.split("/")[-1].split("_")[0]
    print(species)
    missing = 0
    total =0 
    output = open("../data/gene_duplicates/{}.paralog_divergence.tsv".format(species), 'w')
    genes =  CDSs[species]
    for line in open(paralog_file).readlines()[1:]:
        pair = line.strip().split()
        cds1, cds2 = genes[pair[0]], genes[pair[1]]
        with open('../data/gene_duplicates/translation_alignments/tmp.unaligned.fasta','w') as o:
            o.write(">{}\n{}\n".format(pair[0], cds1.seq)) 
            o.write(">{}\n{}\n".format(pair[1], cds2.seq))
        try:
            aligned = translation_aligner('../data/gene_duplicates/translation_alignments/tmp.unaligned.fasta', "../data/gene_duplicates/translation_alignments/translation.tmp".format(species), write=True)
            total+=1
        except Exception as msg:
            print("\tPair {} timed out!\n".format(pair) )
            missing +=1
            total+=1
            continue
        k0,k4, total_sites = k0k4('../data/gene_duplicates/translation_alignments/translation.tmp.aln.fasta')
        data = [pair[0], pair[1], total_sites, k0, k4]
        output.write("\t".join([str(i) for i in data]) + "\n")
        
        if bc > 2:
            pass
        bc +=1
    output.close()
    print("\t", missing, total)

CREN
	 0 297
CINC
	 0 142
CSCH
	 0 134
CMAL
	 0 114
JG5
	 0 169
